In [ ]:
import alias
import alias.injection as inj

import numpy as np
import matplotlib.pyplot as plt

import scipy

import tqdm.autonotebook as tqdm

import math

In [ ]:
median = np.nanmedian(ds.flux, axis=1)
residuals = ds.flux - median[:,None]

In [ ]:
# Manually inspecting small regions of the spectrum to identify continuum pixels.

_ = plt.plot(ds.flux.T[:,:50], color='black', alpha=0.2)

n = 41
plt.xlim(200*n, 200*(n+1))
plt.ylim(0.5, 1.5)

Through visual inspection of the spectra, the continuum pixels were selectet to be at indices:

- CCD 1: 250, 450, 760, 860, 1050, 1300, 1500, 1700, 1885, 2157, 2300, 2460, 2690, 2900, 3100, and 3270
- CCD 2: 3555, 3675, 3900, 4150, 4260, 4500, 4700, 4850, 5030, 5360, 5460, 5675, 5865, and 6100
- CCD 3: 6365, 6490, 6730, 6875, 7050, 7370, 7500, 7700, 7950, 8075, and 8250

and the gaps between the CCDs were determined to be at indices 3400 and 6250.

In [ ]:
continuum_pix = [
    [ 250, 450, 760, 1050, 1300, 1700, 1885, 2300, 2460, 2690, 2900, 3270 ],
    [ 3555, 3900, 4150, 4260, 4700, 4850, 5030, 5460, 5675, 5865, 6100 ],
    [ 6365, 6490, 6730, 6875, 7050, 7370, 7500, 7700, 7950, 8075, 8250 ]
]

def continuum_norm(flux, ivar):
    
    p = [ np.polyfit(cont_p, flux[cont_p], 6) for cont_p in continuum_pix]
    cont = np.array([ np.polyval(c, range(len(ds.wave))) for c in p])
    
    flux_norm = np.copy(flux)
    ivar_norm = np.copy(ivar)
    
    flux_norm[:3400] /= cont[0][:3400]
    flux_norm[3400:6250] /= cont[1][3400:6250]
    flux_norm[6250:] /= cont[2][6250:]

    ivar_norm[:3400] *= cont[0][:3400]**2
    ivar_norm[3400:6250] *= cont[1][3400:6250]**2
    ivar_norm[6250:] *= cont[2][6250:]**2

    return flux_norm, ivar_norm

In [ ]:
plt.figure(figsize=(12,4))

plt.plot(ds.flux[101])
plt.plot([3400]*2, (0,2))
plt.plot([6250]*2, (0,2))
plt.ylim(0.5, 1.3)

plt.xlabel(r'$\lambda$ ($\AA$)')
plt.ylabel(r'Relative Flux')


plt.xlabel(r'$\lambda$ ($\AA$)')
_ = plt.ylabel(r'Relative Flux')

plt.tight_layout()

In [ ]:
data = np.array([ continuum_norm(ds.flux[n], ds.ivar[n]) for n in range(len(ds.flux))])

flux_norm = data[:,0,:]
ivar_norm = data[:,1,:]

In [ ]:
plt.figure(figsize=(12,4))

plt.plot(ds.wave, flux_norm[101])
plt.ylim(0.5, 1.3)


plt.xlabel(r'$\lambda$ ($\AA$)')
_ = plt.ylabel(r'Relative Flux')

plt.tight_layout()

plt.savefig('continuum_normalized.png', dpi=150)

In [ ]:
plt.figure(figsize=(12,4))

spec = 103

plt.plot(ds.wave, ds.flux[spec])

plt.xlabel(r'$\lambda$ ($\AA$)')
plt.ylabel(r'Relative Flux')

_ = plt.tight_layout()

In [ ]:
# Split the flux array into many 200-pixel segments

segment_len = 100

flux = np.concatenate((ds.flux[spec], [math.nan]*(segment_len - len(ds.wave) % segment_len)))

print(len(ds.wave))
print(len(flux))

f_idx = np.array([ range(segment_len*n, segment_len*n+segment_len) for n in range(0, int(len(flux)/segment_len)) ])
flux_segments = flux[f_idx]

print(np.shape(flux_segments))

plt.figure(figsize=(12,4))
plt.plot(flux_segments.T)
plt.xlabel(r'$\Delta\lambda$ (Pixels)')
plt.ylabel(r'Relative Flux')

In [ ]:
# Get pixels in percentile range 80-90
max_perc = np.nanpercentile(flux_segments, 90, axis=1)
min_perc = np.nanpercentile(flux_segments, 70, axis=1)

print(np.shape(max_perc))
print(np.shape(min_perc))

plt.figure(figsize=(12,4))
plt.plot(max_perc, label='Continuum maximum')
plt.plot(min_perc, label='Continuum minimum')
plt.xlabel(r'Segment Index')
plt.ylabel(r'Relative Flux')
plt.legend()

In [ ]:
# Plot percentile range for sample segment to check it is good for isolating continuum pixels
n = 62

continuum_mask = (flux_segments[n] > min_perc[n]) & (flux_segments[n] < max_perc[n])

plt.figure(figsize=(12,4))
plt.plot((0,segment_len), [max_perc[n]]*2, label='Continuum Maximum')
plt.plot((0,segment_len), [min_perc[n]]*2, label='Continuum Minimum')
plt.plot(flux_segments[n], label='Spectrum')

plt.scatter(np.array(range(0, segment_len))[continuum_mask], flux_segments[n][continuum_mask], label='Continuum Pixels')
plt.xlabel(r'Pixels')
plt.ylabel(r'Relative Flux')
plt.legend()

In [ ]:
# Combine the continuum masks for all individual segments to collect all continuum pixels

all_continuum_pixels = [ np.where((flux_segments[n] > min_perc[n]) & (flux_segments[n] < max_perc[n]))[0] + segment_len*n for n in range(len(flux_segments)) ]
all_continuum_pixels = np.concatenate(all_continuum_pixels)

print(np.shape(all_continuum_pixels))
print(all_continuum_pixels)


plt.figure(figsize=(12,4))

plt.scatter(ds.wave[all_continuum_pixels], ds.flux[spec][all_continuum_pixels], label='Continuum Pixels', color='red', s=5)
plt.plot(ds.wave, ds.flux[spec], label='Spectrum')

plt.xlabel(r'$\lambda$ ($\AA$)')
plt.ylabel(r'Relative Flux')
plt.legend()

In [ ]:
# Perform continuum fits for all CCDs
cont1 = all_continuum_pixels[(all_continuum_pixels < 3400)]
fit1 = np.polyfit(cont1, ds.flux[spec][cont1], 6)

cont2 = all_continuum_pixels[(all_continuum_pixels > 3400) & (all_continuum_pixels < 6250)]
fit2 = np.polyfit(cont2, ds.flux[spec][cont2], 6)

cont3 = all_continuum_pixels[(all_continuum_pixels > 6250)]
fit3 = np.polyfit(cont3, ds.flux[spec][cont3], 6)

print(fit1)
print(fit2)
print(fit3)

cont1 = np.polyval(fit1, range(len(ds.wave)))
cont2 = np.polyval(fit2, range(len(ds.wave)))
cont3 = np.polyval(fit3, range(len(ds.wave)))

In [ ]:
# Plot the fits to verify they are acceptable


plt.figure(figsize=(12,4))

plt.plot(ds.wave, ds.flux[spec], label='Spectrum')
plt.plot(ds.wave, cont1, label='Fit 1')
plt.plot(ds.wave, cont2, label='Fit 2')
plt.plot(ds.wave, cont3, label='Fit 3')

plt.ylim(0.4, 1.4)

plt.xlabel(r'$\lambda$ ($\AA$)')
plt.ylabel(r'Relative Flux')
plt.legend()

In [ ]:
# Combine the individual fits into one continuum and plot again
continuum = np.concatenate((cont1[:3400], cont2[3400:6250], cont3[6250:]))

print(np.shape(continuum))

plt.figure(figsize=(12,4))

plt.plot(ds.wave, ds.flux[spec], label='Spectrum')
plt.plot(ds.wave, continuum, label='Continuum Fit')

plt.ylim(0.4, 1.4)

plt.xlabel(r'$\lambda$ ($\AA$)')
plt.ylabel(r'Relative Flux')
plt.legend()

In [ ]:
# Finally perform the continuum normalization and plot the result

normalized_flux = ds.flux[spec] / continuum


plt.figure(figsize=(12,4))

plt.plot(ds.wave, normalized_flux, label='Continuum-Normalized Spectrum')

plt.xlabel(r'$\lambda$ ($\AA$)')
plt.ylabel(r'Relative Flux')
plt.legend()

In [ ]:
# Plot original spectrum to compare

plt.figure(figsize=(12,4))

plt.plot(ds.wave, ds.flux[spec], label='Spectrum')

plt.ylim(0.4, 1.4)

plt.xlabel(r'$\lambda$ ($\AA$)')
plt.ylabel(r'Relative Flux')
plt.legend()

In [ ]:
# Combine the entire process above into a simple function to continuum normalize a single spectrum
def _cn(flux, segment_len=100):
    
    flux = np.concatenate((flux, [math.nan]*(segment_len - len(flux) % segment_len)))
    
    f_idx = np.array([ range(segment_len*n, segment_len*n+segment_len) for n in range(0, int(len(flux)/segment_len)) ])
    flux_segments = flux[f_idx]
    
    max_perc = np.nanpercentile(flux_segments, 80, axis=1)
    min_perc = np.nanpercentile(flux_segments, 70, axis=1)
    
    all_continuum_pixels = np.concatenate([ 
        np.where((flux_segments[n] > min_perc[n]) & (flux_segments[n] < max_perc[n]))[0] + segment_len*n 
        for n in range(len(flux_segments))
    ])

    cont1 = all_continuum_pixels[(all_continuum_pixels < 3400)]
    fit1 = np.polyfit(cont1, flux[cont1], 6)
    cont2 = all_continuum_pixels[(all_continuum_pixels > 3400) & (all_continuum_pixels < 6250)]
    fit2 = np.polyfit(cont2, flux[cont2], 6)
    cont3 = all_continuum_pixels[(all_continuum_pixels > 6250)]
    fit3 = np.polyfit(cont3, flux[cont3], 6)
        
    cont1 = np.polyval(fit1, range(len(flux)))
    cont2 = np.polyval(fit2, range(len(flux)))
    cont3 = np.polyval(fit3, range(len(flux)))
    
    continuum = np.concatenate((cont1[:3400], cont2[3400:6250], cont3[6250:]))

    return continuum

# Function to continuum normalize a list of spectra
def continuum_normalize(flux, ivar):
    continuums = np.array([ _cn(f) for f in tqdm.tqdm(flux) ])[:,range(len(flux[0]))]
    return flux/continuums, ivar*continuums**2

In [ ]:
norm_flux, norm_ivar = continuum_normalize(ds.flux, ds.ivar)

In [ ]:

plt.figure(figsize=(12,4))

_ = plt.plot(ds.wave, norm_flux.T, alpha=0.02,  color='black')
plt.ylim(0.4, 1.2)